In [7]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.stem import RSLPStemmer

In [8]:
conn = sqlite3.connect('db/RSS.sqlite')

In [115]:
df = pd.read_sql_query("SELECT * FROM RSS WHERE keyword in ('brasil','economia', 'mundo', 'tecnologia', 'politica', 'esporte')", conn)

In [116]:
df.groupby('publisher').size()

publisher
A_GAZETA_ACRE          10
A_TRIBUNA              14
BAND                  508
BBC                    20
CORREIO_POVO           20
EBC                    59
Folha                 635
G1                   4080
INFOMONEY             636
JORNAL_COMERCIO       226
JORNAL_PIRACICABA      19
O_GLOBO                72
R7                   2521
UOL                  1201
Valor                 703
dtype: int64

In [49]:
pd.set_option('display.max_rows', 1000)
df.groupby('keyword').size()

keyword
F5                            14
acre                          55
agro                         244
alagoas                       58
amapa                         54
amazonas                      63
arte                         258
bahia                         40
bizzaro                       40
brasil                       356
carreira                      83
carreiras                     45
carros                        97
ceara                         38
ciencia                       32
ciencia e saude               93
cinema                       220
comida                        33
distrito_federal              39
economia                     910
educacao                     150
emprego                      200
empresas                     176
equilibrio                    12
espirito_santo                39
esporte                     1542
geral                        669
goias                         35
jogos                         16
loterias                      47
ma

In [42]:
df

,id,keyword,publisher,title
0,2438,brasil,G1,"Carlinhos Maia apresenta show de humor 'Mas, C..."
1,2439,brasil,G1,Michele Maycoth interpreta sucessos de Maria G...
2,2440,brasil,G1,Exposição 'Minha Aldeia' reúne fotografias ins...
3,2441,brasil,G1,Peça teatral amapaense vai narrar aconteciment...
4,2442,brasil,G1,Levantamento do G1 mostra variação de preço na...
5,2443,brasil,G1,Show de humor em Macapá leva histórias de ribe...
6,2444,brasil,G1,"Sem dinheiro, venezuelanos acampam às margens ..."
7,2445,brasil,G1,Barracas de apoio aos romeiros começam a funci...
8,2446,brasil,G1,Quilo do Pirarucu é vendido em Rondônia por R$...
9,2447,brasil,G1,Confira as vagas de emprego do Sine em Macapá ...


In [ ]:
def Tokenize(sentence):
    sentence = sentence.lower()
    sentence = nltk.word_tokenize(sentence)
    return sentence

In [119]:
def RemoveStopWords(sentence):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    phrase = []
    for word in sentence:
        if word not in stopwords:
            phrase.append(word)
    return phrase

In [120]:
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

In [121]:
frase1 = df.title[0]
frase2 = df.title[2]
print(frase1)
print(frase2)

Carlinhos Maia apresenta show de humor 'Mas, Carlos!' pela primeira vez no AP
Exposição 'Minha Aldeia' reúne fotografias inspiradas no cotidiano amapaense


In [122]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /Users/Felipe/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [123]:
def Train():
    training_data = []
    for row in df.iterrows():
        training_data.append({"classe":row[1][1], "frase":row[1][3]})
    print("%s frases incluidas" % len(training_data))
    return training_data

In [124]:
dados = Train()

1505 frases incluidas


In [125]:
def Learning(training_data):
    corpus_words = {}
    for data in training_data:
        frase = data['frase']
        frase = Tokenize(frase)
        frase = Stemming(frase)
        frase = RemoveStopWords(frase)
        class_name = data['classe']
        if class_name not in list(corpus_words.keys()):
            corpus_words[class_name] = {}
        for word in frase:
            if word not in list(corpus_words[class_name].keys()):
                corpus_words[class_name][word] = 1
            else:
                corpus_words[class_name][word] += 1
    return corpus_words

In [126]:
dados = Learning(dados)
print(dados)

{'brasil': {'carl': 2, 'mai': 3, 'apresent': 3, 'show': 2, 'hum': 2, "'ma": 1, ',': 32, '!': 1, "'": 15, 'pel': 7, 'prim': 2, 'vez': 1, 'ap': 4, 'michel': 1, 'maycoth': 1, 'interpret': 1, 'sucess': 1, 'mar': 1, 'gadú': 1, 'noit': 1, 'român': 1, 'expos': 1, "'minh": 2, 'alde': 1, 'reún': 4, 'fotograf': 1, 'inspir': 2, 'cotidi': 1, 'amapa': 2, 'peç': 1, 'teatr': 3, 'vai': 6, 'narr': 1, 'acontec': 1, '19': 1, 'ano': 6, 'últ': 2, 'livr': 1, 'harry': 1, 'pott': 1, 'levant': 1, 'g1': 1, 'mostr': 4, 'vari': 1, 'preç': 1, 'opç': 2, 'transport': 1, 'divinópolil': 2, ';': 3, 'conf': 2, 'macap': 3, 'lev': 1, 'histór': 2, 'ribe': 1, 'amazôn': 2, 'par': 27, 'dinh': 3, 'venezuel': 1, 'acamp': 1, 'marg': 3, 'rodov': 1, 'fronteir': 1, 'brasil': 7, ':': 17, "'aqu": 1, 'menos': 1, 'barrac': 1, 'apoi': 1, 'rom': 1, 'começ': 3, 'funcion': 1, 'nest': 6, 'sext': 1, 'br-365': 1, 'pat': 1, 'min': 4, 'quil': 1, 'pirarucu': 1, 'é': 15, 'vend': 2, 'rondôn': 2, 'r': 3, '$': 4, '9,36': 1, 'val': 6, 'médi': 1, 'vag

In [127]:
def calculate_score(sentence):
    high_score = 0
    classname = 'default'
    for classe in dados.keys():
        pontos = 0
        pontos = calculate_class_score(sentence,classe)
        if pontos > high_score:
            high_score = pontos
            classname = classe
    return classname,high_score

In [128]:
def calculate_class_score(sentence,class_name):
    score = 0 
    sentence = Tokenize(sentence)
    sentence = Stemming(sentence)
    for word in sentence:
        if word in dados[class_name]:
            score += dados[class_name][word]
    print(score,class_name)
    return score

In [129]:
print(calculate_score('O PIB é a soma de todos os bens e serviços produzidos no país e serve para medir a evolução da economia.'))

51 brasil
86 economia
101 mundo
25 politica
110 tecnologia
101 esporte
('tecnologia', 110)
